# Sweep pytorch tutorial

📢 해당 노트북은 Wandb 공식 홈페이지 중 다음 내용을 참고했습니다.
  
[Organizing Hyperparameter Sweeps in PyTorch with W&B](https://colab.research.google.com/github/wandb/examples/blob/master/colabs/pytorch/Organizing_Hyperparameter_Sweeps_in_PyTorch_with_W%26B.ipynb#scrollTo=6IgMNFFT2qe_)  
[Wandb Document, Define seep configuration](https://docs.wandb.ai/guides/sweeps/define-sweep-configuration)


### 목차  
Step 0: W&B Setup하기  
Step 1: Congfig 정의하기  
Step 2: Dataloader 함수 정의하기  
Step 3: 모델 정의하기  
Step 4: Train 함수와 Vaild 함수 정의하기  
Step 5: Sweep 실행하기  

## Step 0: W&B Setup하기
colab에서 WandB를 사용하려면 wandb modul을 install 해야합니다.

In [ ]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch
import torch.nn as nn
import torch.optim as optim
import wandb
import math
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
%%capture
!pip install wandb --upgrade

In [ ]:
import wandb

wandb.login()

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

## Step1. Config 정의하기

sweep에 필요한 config 파일을 정의합니다.  

그 외 config 정의 방법들 .. https://docs.wandb.ai/guides/sweeps/define-sweep-configuration

In [ ]:
sweep_config = {
    'name' : 'bayes-test',
    'method': 'random',
    'metric' : {
        'name': 'val_loss',
        'goal': 'minimize'
        },
    'parameters' : {
        'optimizer': {
            'values': ['adam', 'sgd']
            },
        'dropout': {
            'values': [0.3, 0.4]
            },
        'learning_rate': {
            'distribution': 'uniform',
            'min': 0,
            'max': 0.1
            },
        'epochs': {
            'values': [5, 6]
            },
        'batch_size': {
            'distribution': 'q_log_uniform',
            'q': 1,
            'min': math.log(32),
            'max': math.log(256),
            }
        }
    }

## Step2. Dataloader 함수 정의하기
mnist 함수를 사용해서 dataloader를 구성하겠습니다.

In [ ]:
def SweepDataset(batch_size):
    transform = transforms.Compose(
        [transforms.ToTensor(),
         transforms.Normalize((0.1307,), (0.3081,))])

    train_data = datasets.MNIST(".",
                train=True,
                download=True,
                transform=transform)

    test_data = datasets.MNIST(".",
                train=False,
                download=True,
                transform=transform)


    train_loader = DataLoader(train_data, batch_size=batch_size)
    test_loader = DataLoader(test_data, batch_size=batch_size)

    return train_loader, test_loader

## Step3. 모델 정의하기
모델과 optimizer를 정의해줍니다.

In [ ]:
class ConvNet(nn.Module):
    def __init__(self, dropout):
        super(ConvNet, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 32, 3, 1, 1), nn.ReLU(),
            nn.MaxPool2d(2, 2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(32, 64, 3, 1, 1), nn.ReLU(),
            nn.MaxPool2d(2, 2))
        self.layer3 = nn.Sequential(
            nn.Linear(64 * 7 * 7, 128, bias=True), nn.ReLU(),
            nn.Dropout2d(p=dropout))
        self.layer4 = nn.Sequential(
            nn.Linear(128, 84), nn.ReLU(),
            nn.Dropout2d(p=dropout))
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = x.view(x.size(0),-1)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.fc3(x)
        return x

In [ ]:
def build_optimizer(network, optimizer, learning_rate):
    if optimizer == "sgd":
        optimizer = optim.SGD(network.parameters(),
                              lr=learning_rate, momentum=0.9)
    elif optimizer == "adam":
        optimizer = optim.Adam(network.parameters(),
                               lr=learning_rate)
    return optimizer

## Step4. Train 함수와 Vaild함수 정의하기
train함수는 학습을 진행하면서 loss를 출력합니다.  
vaild함수는 학습 중간에 Accuracy를 사용해서 평가합니다.

In [ ]:
def train(model, loader, criterion, optimizer, device, config, wandb):
    model.train()
    wandb.watch(model, criterion, log="all", log_freq=10)

    for epoch in range(config.epochs):
        cumu_loss = 0
        for images, labels in loader:
            images, labels  = images.to(device), labels.to(device)

            output = model(images)
            loss = criterion(output, labels)
            cumu_loss += loss.item()

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        avg_loss = cumu_loss / len(loader)
        wandb.log({"train_loss": avg_loss}, step=epoch)
        print(f"TRAIN: EPOCH {epoch + 1:04d} / {config.epochs:04d} | Epoch LOSS {avg_loss:.4f}")

In [ ]:
def vaild(model, loader, criterion, device,  wandb):
    model.eval()

    with torch.no_grad():
        correct, test_loss = 0, 0
        for data, target in loader:
            data, target = data.to(device), target.to(device)
            output = model(data)

            test_loss += criterion(output, target).item()

            pred = output.max(1, keepdim=True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()


    val_loss = test_loss / len(loader)
    print(f"VALID: LOSS {val_loss:.4f} | Accuracy {val_loss:.4f} ")
    wandb.log({
        "val_acc": 100. * correct / len(loader.dataset),
        "val_loss": val_loss})

## Step5. Sweep 실행하기
- `wandb.init(config)` : hyperparameter의 초기값을 wandb.init에 입력으로 넣어줌
- `w_config = wandb.config` : sweep할 대상인 hyperparameter를 config로 정의
- `w_config.optimizer` : loader, model, optimizer에 `w_config`를 매개변수로 전달
- `wandb.watch(model, log='all')` : wandb.watch함수로 gradient추적
- `wandb.log({'Epoch':epoch, 'loss':avg_loss, 'epoch':epoch})` : epoch별로 나오는 log를 wandb.log에 저장
- `sweep_id = wandb.sweep(config.sweep_config)` : config파일에 정의해둔 구성요소를 wandb.sweep에 입력  
    - [command line](https://docs.wandb.ai/ref/cli/wandb-sweep)

    - [python code](https://docs.wandb.ai/ref/python/sweep)

    - [github](https://github.com/wandb/wandb/blob/latest/wandb/sdk/wandb_sweep.py#L31-L118)
- `wandb.agent(sweep_id, train, count=2)` : sweep_id, train함수, count횟수 (몇 번 sweep을 진행할지)를 wandb.agent에 입력하고, sweep을 실행
    - [command line](https://docs.wandb.ai/ref/cli/wandb-agent)

    - [python](https://docs.wandb.ai/ref/python/agent)

    - [github](https://github.com/wandb/wandb/blob/latest/wandb/wandb_agent.py#L589-L650)

In [ ]:
def run_sweeep(config=None):
    wandb.init(config=config)

    w_config = wandb.config

    criterion = nn.CrossEntropyLoss()
    train_loader, vaild_loader = SweepDataset(w_config.batch_size)
    model = ConvNet(w_config.dropout).to(DEVICE)
    optimizer = build_optimizer(model, w_config.optimizer, w_config.learning_rate)

    train(model, train_loader, criterion, optimizer, DEVICE, w_config, wandb)
    vaild(model, vaild_loader, criterion, DEVICE, wandb)

### entity와 project에 대해서

entitiy와 project는 agent뿐만 아니라 wandb.sweep에서도 설정할 수 있고 그 이전에 sweep_config에서도 설정할 수도 있습니다.

최종단계인 agent에 입력했지만 원하는대로 커스텀하시기 바랍니다.

단, entitiy와 project를 입력하는 것 자체는 꼭 기억하시기 바랍니다.

In [ ]:
sweep_id = wandb.sweep(sweep_config, project="sweep_tutorial", entity='rimiyeyo')
wandb.agent(sweep_id, run_sweeep, count=6)


경우에 따라(parameter의 조합이 무한대일때) sweep이 종료가 되지 않을 수 있습니다.

ctrl + c 로(여기에선 실행 중단)으로 종료해주셔야합니다.

In [ ]:
wandb.finish()